In [ ]:
import os 
os.chdir('/Users/wangjialong/Documents/code/saic_project/global-vehicle-dragon/data-processing')

from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained("shibing624/text2vec-base-chinese")
model = AutoModel.from_pretrained("shibing624/text2vec-base-chinese")

In [ ]:
# 准备输入文本
text = \
'''
Global DCS-Vehicle : 全球整车发票 - GV Invoice

1. [Global DCS-Vehicle](index.html)
2. [全球整车系统设计 - System Design](1050081.html)
3. [通用语言 - Ubiquitous Language](1051364.html)
4. [财务UL - Finance UL](1052871.html)

 Global DCS-Vehicle : 全球整车发票 - GV Invoice
===========================================

 Created by  WangJialong 王佳龙, last modified on 11月 20, 2023

| 版本 | [Sprint 7 - Germany](https://confluence.smil.com:17443/display/GLBDCSVE/Sprint+7+-+Germany) |
| 类型 | [层超类型](8323267.html#id-领域驱动设计中的通用语言-superlayer) |
| 作者 | [WangJialong 王佳龙](https://confluence.smil.com:17443/display/~jialong.wang)  |
| 更新时间 | 2023-11-06 |

属性 - Attribute
==============

| 属性 | 类型 | 描述 | 非空 | 限制 |
| --- | --- | --- | --- | --- |
| InvoiceNo | [系统发票号 - Invoice No](8323282.html) | 系统发票号 | M | 无 |
| BizInvoiceNo | [业务发票号 - Biz Invoice No](8323291.html) | 业务发票号 | - | 无 |
| InvoiceStatus | [发票状态 - Invoice Status](8323349.html) | 发票状态 | M | 枚举 |
| Date | Instant | 开票日期 | M | 不能创建未来日期的发票 |
| CancelDate | LocalDate | 取消开票日期 | - | 发票状态为Cancelled，才可以有该字段 |
| Seller | [发票卖方 - Invoice Seller](9306233.html) | 开票方信息 | M |  |
| Buyer | [发票买方 - Invoice Buyer](9306497.html) | 发票买方的信息 | M |  |
| InvoiceInfo | [发票信息 - InvoiceInfo](9306665.html) | 发票创建和取消的信息 | - |  |

功能 - Function
=============

### 取消发票 - cancelInvoice

战术设计图
=====

![](attachments/1052881/9306135.png)

Attachments:
------------

![](images/icons/bullet_blue.gif)

'''
encoded_input = tokenizer(text, return_tensors='pt')
# print(encoded_input)

In [ ]:
from langchain.text_splitter import MarkdownTextSplitter

markdown_spilitter = MarkdownTextSplitter(chunk_size=512, chunk_overlap=0)
docs = markdown_spilitter.create_documents([text])

# for split in docs:
#     print(split.page_content)
first = docs[0].page_content
print(first)



In [ ]:
from towhee import AutoPipes, AutoConfig


config = AutoConfig.load_config('sentence_embedding')
# config.model = 'paraphrase-albert-small-v2'
# config.model = 'all-MiniLM-L6-v2'

sentence_embedding = AutoPipes.pipeline('sentence_embedding', config=config)
embedding = sentence_embedding('how are you?').get()
print(embedding)

In [31]:
from towhee import AutoPipes, AutoConfig

first = '''
Global DCS-Vehicle : 全球整车发票 - GV Invoice

1. [Global DCS-Vehicle](index.html)
2. [全球整车系统设计 - System Design](1050081.html)
3. [通用语言 - Ubiquitous Language](1051364.html)
4. [财务UL - Finance UL](1052871.html)

 Global DCS-Vehicle : 全球整车发票 - GV Invoice
===========================================

 Created by  WangJialong 王佳龙, last modified on 11月 20, 2023
'''

# 默认模型 all-MiniLM-L6-v2
config = AutoConfig.load_config('sentence_embedding')
# config.model = 'all-MiniLM-L6-v2'
# help(sentence_embedding)

# generate embedding
sentence_embedding = AutoPipes.pipeline('sentence_embedding', config=config)
embedding = sentence_embedding(first).get()
# print(embedding)

insert_conf = AutoConfig.load_config('insert_milvus')
insert_conf.collection_name = 'dkb_v1'
insert_pipe = AutoPipes.pipeline('insert_milvus', insert_conf)
# insert text and embedding into Milvus
insert_pipe(["test1", embedding])
                                

2024-02-24 01:26:07,086 - 123146063142912 - decorators.py-decorators:134 - ERROR: RPC error: [batch_insert], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 1)>, <Time:{'RPC start': '2024-02-24 01:26:07.086237', 'RPC error': '2024-02-24 01:26:07.086284'}>


RuntimeError: Node-ann-insert/milvus-client-0 runs failed, error msg: <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 1)>, Traceback (most recent call last):
  File "/Users/wangjialong/anaconda3/envs/data-processing/lib/python3.9/site-packages/towhee/runtime/nodes/node.py", line 158, in _call
    return True, self._op(*inputs), None
  File "/Users/wangjialong/.towhee/operators/ann-insert/milvus-client/versions/main/milvus_client.py", line 52, in __call__
    mr = self._collection.insert(row)
  File "/Users/wangjialong/anaconda3/envs/data-processing/lib/python3.9/site-packages/pymilvus/orm/collection.py", line 497, in insert
    res = conn.batch_insert(
  File "/Users/wangjialong/anaconda3/envs/data-processing/lib/python3.9/site-packages/pymilvus/decorators.py", line 135, in handler
    raise e from e
  File "/Users/wangjialong/anaconda3/envs/data-processing/lib/python3.9/site-packages/pymilvus/decorators.py", line 131, in handler
    return func(*ar

In [ ]:
from towhee import AutoPipes, AutoConfig

# set MilvusSearchConfig for the built-in search_milvus pipeline
search_conf = AutoConfig.load_config('search_milvus')
search_conf.collection_name = 'dkb_v1'
search_conf.search_params = {'output_fields': ['id', 'name']}

search_pipe = AutoPipes.pipeline('search_milvus', search_conf)

# generate embedding
embedding = sentence_embedding('全球整车系统设计').get()[0]

# search embedding and get results in Milvus
search_pipe(embedding).get_dict()

In [ ]:
# 获取模型的输出
with torch.no_grad():
    model_output = model(**encoded_input)


In [ ]:
# 获取文本的向量表示
text_vector = model_output.last_hidden_state.mean(dim=1)
print(text_vector)